In [ ]:
!pip install livelossplot

In [ ]:
import keras,tensorflow
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D , Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt 
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
import math

from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from livelossplot import PlotLossesKerasTF

from tensorflow.keras.optimizers import SGD

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weight = dict()
    
    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0
    
    return class_weight

In [ ]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="../input/fer2013/train",
                                       target_size=(48,48),
                                       color_mode='grayscale',
                                       batch_size= 64,
                                       class_mode = "categorical"
                                       )
tsdata = ImageDataGenerator(validation_split=0.5)
validation = tsdata.flow_from_directory(directory="../input/fer2013/test", #change path here accordingly
                                      target_size=(48,48),
                                      batch_size = 64,
                                      color_mode = "grayscale",
                                      class_mode = "categorical",
                                      shuffle=False,
                                      subset='training'
                                     )
testdata = tsdata.flow_from_directory(directory="../input/fer2013/train", 
                                      target_size=(48,48),
                                      batch_size = 64,
                                      color_mode = "grayscale",
                                      class_mode = "categorical",
                                      shuffle=False,
                                      subset='validation'
                                     )

In [ ]:
class_weights = create_class_weight(Counter(traindata.classes))
print(class_weights)

In [ ]:
model = Sequential()

model.add(BatchNormalization(input_shape=(48,48,1)))
model.add(Conv2D(32, (3, 3), activation='relu',padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))         
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(7, activation='softmax'))

In [ ]:
checkpoint = ModelCheckpoint('model.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              factor=0.5, 
                              patience=2, 
                              min_lr=0.00001, 
                              verbose=1)

callbacks = [PlotLossesKerasTF(),checkpoint,reduce_lr]

In [ ]:
sgd = SGD(lr=0.01,momentum=0.9, decay=0.0001, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [ ]:
history=model.fit_generator(generator=traindata,
                            callbacks=callbacks,
                            validation_data=validation,
                            class_weight=class_weights, 
                            epochs=30)

In [ ]:
#save model architecture to JSON (for the Flask app)
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_weights.h5")

In [ ]:
import time
from keras.preprocessing import image
img = image.load_img("../input/testphoto/testphoto7.png",target_size=(48,48),grayscale=True)
img = np.asarray(img)
plt.imshow(img,cmap='gray')
img=img.reshape(48,48,1)
img = np.expand_dims(img, axis=0)
start_time = time.time()
output = lmodel.predict(img)
print("--- %s seconds ---" % (time.time() - start_time))
print(str(output[0][0]*100) +"% angry")
print(str(output[0][1]*100) +"% disgust")
print(str(output[0][2]*100) +"% fear")
print(str(output[0][3]*100) +"% happy")
print(str(output[0][4]*100) +"% neutral")
print(str(output[0][5]*100)+"% sad")
print(str(output[0][6]*100)+"% surprise")


In [ ]:
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('CNN Metrices(Loss)')
plt.ylabel('% value')
plt.xlabel('Epoch')
plt.legend(loc="upper left")
plt.show()
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('CNN Metrices (Accuracy)')
plt.ylabel('% value')
plt.xlabel('Epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
emotions = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Neutral',5:'Sad', 6:'Surprise'}

y_pred = lmodel.predict_generator(testdata).argmax(axis=1)
y_true = testdata.classes

cmat_df_test=pd.DataFrame(
  confusion_matrix(y_true, y_pred, normalize='true').round(2),
  index=emotions.values(), 
  columns=emotions.values()
  )

plt.figure(figsize=(5,5))
sns.heatmap(cmat_df_test,annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.title('Confusion Matrix on Private Test Set')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()